In [2]:
from PIL import Image
import numpy as np 
xRand = {}
yRand = {}
def mapping():
    k_new = (x_new*ncol)+y_new
    k = (x*ncol)+y
    colors_bin_new[k_new,bit] = colors_bin[k,bit]
    #print (colors_bin_new[k_new,bit])
    #print (k_new)
    #print (bit)
# generate randoms numbers x's and y's
def generate():
    xRand[0] = 0.12345678912342
    yRand[0] = 0.87865765433212
    alpha = 3.99999
    
    for i in range(110):
        xRand[i+1]= alpha*xRand[i]*(1-xRand[i]) 
        yRand[i+1]= alpha*yRand[i]*(1-yRand[i]) 
###########
def generate_diffusion():   
    cs[0] = key_d
    for i in range(999999):
        cs[i+1]= alpha2*cs[i]*(1-cs[i]) 
############
im = Image.open("lena.jpeg")
pix = im.load()
#print (im.size)

#coordinate = x,y = 180,180
#print (im.getpixel(coordinate))
size = im.size
print (size)
temp_arr = {}
colors = {} #average of r,g,b for each pixel
colors_bin = {} # binary 8-bit representation of colors values
k=0
for x in range(size[0]):
    for y in range(size[1]):
        #print(pix[x,y])
        temp_arr[k] = pix[x,y]
        temp = temp_arr[k]
        temp2 = (temp[0]+temp[1]+temp[2])/3
        colors[k] = int(temp2)
        temp3 = format (colors[k],'08b')
        for i in range(8):
            colors_bin[k,i] = temp3[i]
        k = k+1
nrow = size[0]
ncol = size[1]
#confusion stage
generate()
colors_bin_new = {}
p = {}
q = {}
for i in range(5):
    p[i] = int(abs(xRand[i+100]*(10**14))%nrow)
    q[i] = int(abs(yRand[i+100]*(10**14))%ncol)

for i in range(5):
    for x in range(size[0]):
        for y in range(size[1]):
            x_new = (x + p[i]*y)%nrow
            y_new = (q[i]*x+(p[i]*q[i]+1)*y)%ncol
            if i==0:
                for bit in range(4):
                    mapping()
            else:
                bit = i+3
                mapping()
#print(colors_bin_new[100,7])              
##diffusion
key_d = 0.34565487923280
alpha2 = 4
cs = {}
generate_diffusion()
c = {}
L = {}
cd = {}
f = {}
print (type(colors_bin_new))
print (type(colors_bin))

total_loop = nrow*ncol
#print("total loop",total_loop, nrow,ncol)
seed_c = int((4*key_d*(1-key_d)*(10**14)) % 256) # c[-1]
seed_cd = seed_c / 1000 # cd[-1]
seed_f = alpha2*seed_cd*(1-seed_cd) #f[-1]
L[0]= int(((seed_f*1000)+(cs[0]*(10**10))%256)%256)

for i in range(total_loop):
    p = 0
    for j in range(8):
        rtemp = 2**(7-j)
        array = np.array(colors_bin_new)
        temp_1 = colors_bin_new[i,j]
        #print(type(temp_1))
        #rtemp2 = list.pop(list(colors_bin_new),[i,j])
        #print(type(array))
        rtemp2 = int(temp_1)        
        p = p + rtemp2*rtemp #converting binary in each row(pixel) to decimal
    c[i] = p ^ L[i]
    cd[i] = c[i]/1000
    f[i] = alpha2*cd[i]*(1-cd[i])
    L[i+1] = int(( (f[i]*1000) + (cs[i+1]*(10**10))%256 )%256)
    #print(i)



(225, 225)
<class 'dict'>
<class 'dict'>


In [59]:
##creating a 2d pixel value matrix where each item has (r,g,b) value of each pixel

encrypted_pixels = [['1' for col in range(nrow)] for col in range(ncol)] 
for i in range(nrow):
    for j in range(ncol):  
        temp_list = [0,0,0]
        temp_list[0] = c[i*ncol+j]
        temp_list[1] = c[i*ncol+j]
        temp_list[2] = c[i*ncol+j]
        encrypted_pixels[i][j] = temp_list
        #print(encrypted_pixels[i,j])

In [58]:
##getting the image from the encrypted order of the pixels 

cmap = plt.cm.jet
plt.imsave('encrypted.png', encrypted_pixels, cmap=cmap) ##encrypted image will be saved in the same folder where the code is present